# stata_session

> A class for representing a Stata session
- order: 8

Some parts adapted from the [stata_kernel version](https://github.com/kylebarron/stata_kernel/blob/master/stata_kernel/completions.py), limited for now to variables, globals, locals, scalars, matrices, and file names.

In [ ]:
#| default_exp stata_session
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.misc_utils import print_red
from nbstata.stata import run_direct, get_local, get_scalar
from nbstata.stata_more import diverted_stata_output_quicker, local_names, run_sfi
from nbstata.stata_more import get_local_dict as _get_local_dict
from nbstata.code_utils import (
    valid_single_line_code,
    ending_sc_delimiter,
    ends_in_comment_block,
    ending_code_version,
)
from nbstata.noecho import run_as_program_w_locals, run_noecho
from fastcore.basics import patch_to
from textwrap import dedent
import re

In [ ]:
from nbstata.config import launch_stata

In [ ]:
#| export
class StataSession():
    def __init__(self):
        """"""
        self.sc_delimiter = False
        self.code_version = None
        self.stata_version = None
        self.clear_suggestions()
        self._compile_re()

    def clear_suggestions(self):
        self.suggestions = None
        
    def _compile_re(self):
        self.matchall = re.compile(
            r"\A.*?"
            r"^%varlist%(?P<varlist>.*?)"
            r"%globals%(?P<globals>.*?)"
            #r"%locals%(?P<locals>.*?)"
            r"%scalars%(?P<scalars>.*?)"
            r"%matrices%(?P<matrices>.*?)%end%", #"(\Z|---+\s*end)",
            flags=re.DOTALL + re.MULTILINE).match

        # Varlist-style matching; applies to most
        self.varlist = re.compile(r"(?:\s+)(\S+)", flags=re.MULTILINE)

        # file-style matching
        self.filelist = re.compile(r"[\r\n]{1,2}", flags=re.MULTILINE)

        # Clean line-breaks.
        self.varclean = re.compile(
            r"(?=\s*)[\r\n]{1,2}?^>\s", flags=re.MULTILINE).sub
        
        #         # Match output from mata mata desc
#         self.matadesc = re.compile(
#             r"(\A.*?---+|---+[\r\n]*\Z)", flags=re.MULTILINE + re.DOTALL)

#         self.matalist = re.compile(
#             r"(?:.*?)\s(\S+)\s*$", flags=re.MULTILINE + re.DOTALL)

#         self.mataclean = re.compile(r"\W.*?(\b|$)")
#         self.matasearch = re.compile(r"(?P<kw>\w.*?(?=\W|\b|$))").search

In [ ]:
#| export
@patch_to(StataSession)
def refresh_suggestions(self):
    self.suggestions = self.get_suggestions()

In [ ]:
#| export
@patch_to(StataSession)
def _completions(self):
    return diverted_stata_output_quicker(dedent("""\
        local _temp_completions_while_local_ = 1
        while `_temp_completions_while_local_' {
        set more off
        set trace off
        if `"`varlist'"' != "" {
        local _temp_completions_varlist_loc_ `"`varlist'"'
        }
        syntax [varlist]
        disp "%varlist%"
        disp `"`varlist'"'
        macro drop _varlist __temp_completions_while_local_
        if `"`_temp_completions_varlist_loc_'"' != "" {
        local varlist `"`_temp_completions_varlist_loc_'"'
        macro drop __temp_completions_varlist_loc_
        }
        disp "%globals%"
        disp `"`:all globals'"'
        *disp "%locals%"
        *mata : invtokens(st_dir("local", "macro", "*")')
        disp "%scalars%"
        disp `"`:all scalars'"'
        disp "%matrices%"
        disp `"`:all matrices'"'
        disp "%end%"
        local _temp_completions_while_local_ = 0
        }
        macro drop _temp_completions_while_local_
    """))

In [ ]:
#| hide
test_instance = StataSession()

In [ ]:
#| hide
#| eval: False
launch_stata(splash=False)
print(test_instance._completions())

%varlist%

%globals%
S_level F1 F2 F7 F8 S_ADO S_StataMP S_StataSE S_CONSOLE S_FLAVOR S_OS S_OSDTL S
> _MACH
%scalars%

%matrices%

%end%



In [ ]:
#| export
@patch_to(StataSession)
def _get_locals(self):
    return self.suggestions['locals'] if self.suggestions else local_names()

In [ ]:
#| hide
from fastcore.test import test_eq

In [ ]:
#| eval: False
run_sfi("""\
local varlist = 5
local varlist1 = 5""")

In [ ]:
#| hide
#| eval: False
test_eq(set(test_instance._get_locals()), {'varlist', 'varlist1'})

In [ ]:
#| export
@patch_to(StataSession)
def get_suggestions(self):
    match = self.matchall(self._completions())
    suggestions = match.groupdict()
#         suggestions['mata'] = self._parse_mata_desc(suggestions['mata'])
#         suggestions['programs'] = self._parse_programs_desc(
#             suggestions['programs'])
    for k, v in suggestions.items():
        suggestions[k] = self.varlist.findall(self.varclean('', v))
    suggestions['locals'] = self._get_locals()
    return suggestions

In [ ]:
#| eval: false
test_instance.refresh_suggestions()
test_instance.suggestions

{'varlist': [],
 'globals': ['S_level',
  'F1',
  'F2',
  'F7',
  'F8',
  'S_ADO',
  'S_StataMP',
  'S_StataSE',
  'S_CONSOLE',
  'S_FLAVOR',
  'S_OS',
  'S_OSDTL',
  'S_MACH'],
 'scalars': [],
 'matrices': [],
 'locals': ['varlist1', 'varlist']}

In [ ]:
#| hide
from nbstata.stata import get_local

In [ ]:
#| hide
#| eval: false
run_sfi(dedent("""
    local local1 = 1
    local local2 "two"
    local local3 `""3""' """))
print(repr(get_local("local1")))
print(repr(get_local("local2")))
print(repr(get_local("local3")))

'1'
'two'
'"3"'


In [ ]:
#| export
@patch_to(StataSession)
def get_local_dict(self):
    return _get_local_dict(self._get_locals())

In [ ]:
#| eval: False
run_sfi('''\
macro drop _all
local test1 "blah blah" ''')
test_instance.clear_suggestions()
test_eq(test_instance.get_local_dict(), {'test1': 'blah blah'})
run_sfi('local test1 ""')

In [ ]:
#| export
@patch_to(StataSession)
def _run_as_program_w_locals(self, std_code):
    """After `break_out_prog_blocks`, run noecho, inserting locals when needed"""
    return run_as_program_w_locals(std_code, local_dict=self.get_local_dict())

In [ ]:
#| hide
#| eval: false
run_sfi(dedent("""
    macro drop _all
    local local1 = 1
    local local2 "two"
    local local3 `""3""' """))
test_instance.clear_suggestions()
test_instance._run_as_program_w_locals("""disp `"`local1' `local2' `local3'"' """)

1 two "3"


In [ ]:
#| hide
#| eval: false
code = '''\
local test1 "blah blah"
local test2 "blah"
'''
test_instance.clear_suggestions()
test_instance._run_as_program_w_locals("""disp `"`local1' `local2' `local3'"' \n""" + code)
test_eq(test_instance.get_local_dict(), 
        {'test2': 'blah',
         'test1': 'blah blah',
         'local1': '1',
         'local2': 'two',
         'local3': '"3"'})

1 two "3"


## dispatch_run

We incorporate `run_noecho` within a `dispatch_run` wrapper that can serve as an alternative to the official `pystata.stata.run` command, supporting any value of the `echo` or `quietly` parameters. The ordinary `run_direct` (for `echo != None`) is also prefaced to manage delimiters and prevent certain quirks of `pystata.stata.run` from biting.

In [ ]:
#| export
def _run_simple(code, quietly=False, echo=False, sc_delimiter=False):
    if sc_delimiter:
        code = "#delimit;\n" + code
    if len(code.splitlines()) == 1:
        code = valid_single_line_code(code)
    run_direct(code, quietly=quietly, inline=not quietly, echo=echo)

#| hide
We remove comments from single-line code to avoid the error that [would otherwise result](https://www.stata.com/python/pystata/stata.html#pystata.stata.run).

In [ ]:
#| eval: false
_run_simple(dedent('''\
    capture program drop ender
    program define ender
        disp "ender output"
    end
    capture program drop display2
    program define display2
        ender
    end
    display2
    '''), quietly=True)

In [ ]:
#| export
_final_delimiter_warning = (
    "Warning: Code cell (with #delimit; in effect) does not end in ';'. "
    "Exported .do script may behave differently from notebook. "
    "In v1.0, nbstata may trigger an error instead of just a warning."
)

In [ ]:
#| export
@patch_to(StataSession)    
def _update_ending_delimiter(self, code):
    self.sc_delimiter = ending_sc_delimiter(code, self.sc_delimiter)
    _final_character = code.strip()[-1]
    _code_missing_final_delimiter = (self.sc_delimiter
                                     and _final_character != ';')
    if _code_missing_final_delimiter:
        print_red(_final_delimiter_warning)

In [ ]:
#| hide
test_instance.sc_delimiter = True
test_instance._update_ending_delimiter('''disp "test output"''')
test_instance.sc_delimiter = False

In [ ]:
#| export
def warn_re_unclosed_comment_block_if_needed(code):
    if ends_in_comment_block(code):
        print_red("Warning: Code cell ends in a comment block without a "
                  "closing '*/'. Exported .do script may behave differently "
                  "from notebook. In v1.0, nbstata may trigger an error "
                  "instead of just a warning."
                 )

In [ ]:
#| hide
warn_re_unclosed_comment_block_if_needed("/*")

In [ ]:
#| hide
#| eval: False
f"{15.1:0.2f}"

'15.10'

In [ ]:
#| export
@patch_to(StataSession)
def _post_run_hook(self, code):
    self.clear_suggestions()
    if self.stata_version is None:
        self.stata_version = f"{get_scalar('c(stata_version)'):0.2f}"
    self.code_version = ending_code_version(code, self.sc_delimiter, self.code_version, self.stata_version)
    self._update_ending_delimiter(code) # after updating code_version (based on starting sc_delimiter)
    warn_re_unclosed_comment_block_if_needed(code)

In [ ]:
#| export
@patch_to(StataSession)
def dispatch_run(self, code, quietly=False, echo=False, noecho=False):
    if self.code_version:
        version_prefix = "capture version " + self.code_version + (";" if self.sc_delimiter else "\n")
        code = version_prefix + code
    if noecho and not quietly:
        run_noecho(code, self.sc_delimiter, run_as_prog=self._run_as_program_w_locals)
    else:
        _run_simple(code, quietly, echo, self.sc_delimiter)
    self._post_run_hook(code)

In [ ]:
#| eval: false
test_instance.dispatch_run(dedent('''\
    capture program drop ender
    program define ender
        disp "ender output"
    end
    capture program drop display2
    program define display2
        ender
    end
    display2
    '''), quietly=True)

In [ ]:
#| eval: false
test_instance.dispatch_run(dedent('''\
    capture program drop ender
    program define ender
        disp "ender output"
    end
    capture program drop display2
    program define display2
        ender
    end
    display2
    '''), noecho=True)



ender output


In [ ]:
#| eval: false
code = dedent('''\
    python:
    print("hello")
    end
    ''')
test_instance.dispatch_run(code, noecho=True)

hello



In [ ]:
#| eval: false
run_noecho(dedent("""\
    disp `"`local1' `local2' `local3'"'
    disp `"`local1' `local2' `local3' `test1'"'
    """), run_as_prog=test_instance._run_as_program_w_locals)

1 two "3"
1 two "3" blah blah


In [ ]:
#| eval: false
code = """\
local local1 "foo"
local local2 "bar"
local abcd "foo bar"
"""
test_instance.clear_suggestions()
run_noecho(code, run_as_prog=test_instance._run_as_program_w_locals)
test_instance.clear_suggestions()
run_noecho(dedent("""\
    disp `"`local1' `local2' `local3'"'
    disp `"`local1' `local2' `local3' `test1'"'
    """), run_as_prog=test_instance._run_as_program_w_locals)

foo bar "3"
foo bar "3" blah blah


In [ ]:
#| eval: false
test_instance.clear_suggestions()
code2 = '''\
display "line continuation " /// commented out
    "comment"'''
test_instance.dispatch_run(code2, noecho=True)

line continuation comment


In [ ]:
#| eval: false
test_instance.clear_suggestions()
code2 = '''\
display "line continuation " /// commented out
    "comment"'''
test_instance.dispatch_run(code2, noecho=True)

line continuation comment


In [ ]:
#| eval: false
test_instance.clear_suggestions()
code2 = '''\
disp c(version)
version 15.1
disp 1'''
test_instance.dispatch_run(code2, noecho=True)
test_instance.dispatch_run('disp c(version)', noecho=True)

17
1
15.1


In [ ]:
#| hide
#| eval: false
test_instance.clear_suggestions()
code2 = '''\
disp c(version)
version 16
disp 1'''
test_instance.dispatch_run(code2)
test_instance.dispatch_run('disp c(version)')


. capture version 15.1

. disp c(version)
15.1

. version 16

. disp 1
1

. 

. capture version 16

. disp c(version)
16

. 


In [ ]:
#| eval: false
test_instance.clear_suggestions()
code2 = '''\
disp c(version)
version 17
disp 1'''
test_instance.dispatch_run(code2)
test_instance.dispatch_run('disp c(version)', echo=True)


. capture version 16

. disp c(version)
16

. version 17

. disp 1
1

. 
. disp c(version)
17


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()